In [ ]:
import voyageai
import time
import re

vo = voyageai.Client(api_key="<your secret key>")
# This will automatically use the environment variable VOYAGE_API_KEY.
# Alternatively, you can use vo = voyageai.Client(api_key="<your secret key>")

In [ ]:
with open('/home/byron/repos/cmix/data/enwik9', 'r') as file:
    enwik9 = file.read()

In [ ]:
articles = enwik9.split("<page>")
enwik9 = ""

In [ ]:
truncate_article = 4000
articles_per_request = 25
first = 0
last = len(articles)
texts = []
m = {}
file1 = open("embeddings-4k.tsv", "a")  # append mode
for i in range(last-first):
    index = i + first
    start = articles[index].find("<text xml:space=\"preserve\">")
    end = articles[index].find("</text>")
    if (start > 0 and end > 0):
        if (articles[index][start+27] == '#' and (articles[index][start+28] == 'R' or articles[index][start+28] == 'r')):
            continue
        title_start = articles[index].find("<title>")
        title_end = articles[index].find("</title>")
        title = articles[index][title_start+7:title_end]
        m[len(texts)] = index
        content = articles[index][start+27:end]
        content = re.sub("{{[^}]*\n[^}]*}}", "", content, flags = re.MULTILINE)
        content = title + "\n" + content[:truncate_article]
        texts.append(content)
    if (len(texts) == articles_per_request):
        print(index)
        result = vo.embed(texts, model="voyage-large-2-instruct", input_type="document")
        time.sleep(1.0)
        for j in range(len(result.embeddings)):
            file1.write(str(m[j]) + "\t" + '\t'.join(map(str,result.embeddings[j])) + "\n")
        m = {}
        texts = []
if (len(texts) > 0):
    result = vo.embed(texts, model="voyage-large-2-instruct", input_type="document")
    for j in range(len(result.embeddings)):
        file1.write(str(m[j]) + "\t" + '\t'.join(map(str,result.embeddings[j])) + "\n")
    m = {}
    texts = []
file1.close()